In [333]:
import pandas as pd
import sqlite3
from autocorrect import Speller
import re
import json

In [337]:
# Подключение к базе данных
db_path = '../db/data.db'
conn = sqlite3.connect(db_path)

# Загрузка конфига
with open('../cfg/config.json', 'r', encoding="utf-8") as file:
    cfg = json.load(file)

In [338]:
# Загрузка данных из каждой таблицы в отдельный DataFrame
person_df = pd.read_sql("SELECT * FROM person", conn)
education_df = pd.read_sql("SELECT * FROM education", conn)
jobs_df = pd.read_sql("SELECT * FROM jobs", conn)

def to_lower(text: str) -> str:
    return text.lower()

def cut(text: str) -> str:
    symbols_for_cut = [",", "(", ";", ":"]
    for symbol in symbols_for_cut:
        idx = text.find(symbol)
        if idx != -1:
            text = text[:idx]
    return text

jobs_df['job_name_norm'] = jobs_df['job_name'].apply(to_lower)
jobs_df['job_name_norm'] = jobs_df['job_name_norm'].apply(cut)

In [340]:
is_spell_check_activated = cfg["activate_spell_check"]
if is_spell_check_activated:
    spell = Speller("ru")
    
    def correct(text: str) -> str:
        return str(spell(text))
    
    jobs_df['job_name_norm'] = jobs_df['job_name_norm'].apply(correct)

In [341]:
jobs_df['job_name_norm'] = jobs_df['job_name_norm'].str.replace(".", " ")
jobs_df['job_name_norm'] = jobs_df['job_name_norm'].str.replace("ё", "е")

# Использование регулярного выражения для замены двух или более пробелов на один пробел
def replace_multiple_spaces(input_string):
    return re.sub(' +', ' ', input_string)

jobs_df['job_name_norm'] = jobs_df['job_name_norm'].apply(replace_multiple_spaces)

# Использование регулярного выражения для замены любого количества табов на один пробел
def replace_tabs_with_space(input_string):
    return re.sub('\t+', ' ', input_string)

jobs_df['job_name_norm'] = jobs_df['job_name_norm'].apply(replace_tabs_with_space)

jobs_df['job_name_norm'] = jobs_df['job_name_norm'].str.replace(r"\s*-\s*", "-", regex=True)
jobs_df['job_name_norm'] = jobs_df['job_name_norm'].str.replace(r"\s*/\s*", " / ", regex=True)


# Словарь для расшифровки сокращений
abbreviation_dict = cfg["abbreviation_dict"]

# Функция для расшифровки сокращений в названиях должностей
def expand_abbreviations(job_name, abbr_dict=abbreviation_dict):
    job_name = " " + job_name + " "
    for abbr, full_form in abbr_dict.items():
        job_name = job_name.replace(" " + abbr + " ", " " + full_form + " ")
    return job_name.strip()

jobs_df['job_name_norm'] = jobs_df['job_name_norm'].apply(expand_abbreviations)


def remove_reducant_space(text: str) -> str:
    return text.strip()

jobs_df['job_name_norm'] = jobs_df['job_name_norm'].apply(remove_reducant_space)


# функция, которая преобразует римские цифры в арабские в строке
def roman_to_arabic(input_string):
    # Roman numeral mapping
    roman_numerals = {
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5,
        'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10,
        'XI': 11, 'XII': 12, 'XIII': 13, 'XIV': 14, 'XV': 15,
        'XVI': 16, 'XVII': 17, 'XVIII': 18, 'XIX': 19, 'XX': 20,
        # Добавьте больше отображений, если нужно
    }

    # Разбиваем входную строку на слова
    words = input_string.split()

    # Заменяем римские цифры на арабские числа
    for i, word in enumerate(words):
        if word.upper() in roman_numerals:
            words[i] = str(roman_numerals[word.upper()])

    # Объединяем слова обратно в строку
    res = ' '.join(words)

    if res != input_string:
        print("roman_to_arabic(): " + input_string + " -> " + res)

    return res

jobs_df['job_name_norm'] = jobs_df['job_name_norm'].apply(roman_to_arabic)

def drop_everything_after_first_digit(input_string: str) -> str:
    for index, char in enumerate(input_string):
        if char.isdigit():
            return input_string[:index]
    return input_string

jobs_df['job_name_norm'] = jobs_df['job_name_norm'].apply(drop_everything_after_first_digit)

pd.set_option('display.max_rows', 10)
jobs_df

roman_to_arabic(): инженер iii категория -> инженер 3 категория


,id,job_name,person_id,desc,start,end,job_name_norm
0,1,Ветеринарный врач,1,Планирование и проведение ветеринарных лечебно...,Август 2018,None,ветеринарный врач
1,2,Ветеринарный врач,1,Ветеринарный врач,Август 2018,Ноябрь 2019,ветеринарный врач
2,3,Тракторист-машинист,2,-работа на тракторе Джон-Дир 8020\n-ТО и ремон...,Август 2014,Октябрь 2014,тракторист-машинист
3,4,Тракторист-машинист,2,-оператор з/у комбайна Джон Дир\n-ремонт и ТО:...,Июль 2013,Сентябрь 2013,тракторист-машинист
4,5,Агроном,3,-,Сентябрь 2018,None,агроном
...,...,...,...,...,...,...,...
4330,4331,"ст инженер, гл. инженер,зам.директора по произ...",1000,"Организация работы. эксплуатация , техническое...",Август 1976,None,старший инженер
4331,4332,Зоотехник,1001,Контроль поедаемости кормов животными.\nКоррек...,Август 2020,None,зоотехник
4332,4333,Зоотехник,1001,Контроль кормления коров.\nЗаготовка сена.\nПр...,Декабрь 2019,Август 2020,зоотехник
4333,4334,Помощник зоотехника по кормлению,1001,Контроль поедаемости кормов животными.\nКоррек...,Сентябрь 2019,Ноябрь 2019,помощник зоотехника по кормлению


In [342]:
from collections import Counter

words = jobs_df['job_name_norm'].str.split().explode()

# Подсчитать частоту каждого слова
word_counts = Counter(words)

# Вывести самые частые слова
most_common_words = word_counts.most_common()

# Печать результатов
print(most_common_words)

[('по', 472), ('главный', 458), ('ветеринарный', 424), ('врач', 394), ('агроном', 348), ('инженер', 297), ('механик', 211), ('начальник', 210), ('и', 153), ('отдела', 136), ('специалист', 134), ('менеджер', 134), ('директор', 127), ('зоотехник', 125), ('заместитель', 119), ('старший', 114), ('мастер', 110), ('машинист', 107), ('водитель', 103), ('участка', 88), ('директора', 83), ('заведующий', 82), ('ведущий', 80), ('технолог', 79), ('инженер-механик', 76), ('руководитель', 72), ('оператор', 71), ('тракторист-машинист', 62), ('управляющий', 55), ('с', 52), ('цеха', 50), ('оборудования', 49), ('производства', 48), ('службы', 47), ('ремонту', 46), ('слесарь', 45), ('помощник', 45), ('сервисный', 44), ('продажам', 42), ('/', 38), ('тракторист', 35), ('погрузчика', 32), ('врача', 32), ('бригадир', 31), ('сотрудник', 31), ('эксплуатации', 31), ('механизатор', 31), ('отделения', 30), ('научный', 30), ('растений', 30), ('региональный', 30), ('представитель', 30), ('экскаватора', 29), ('произ

In [343]:
pd.set_option('display.max_rows', None)
jobs_df['job_name_norm'].value_counts()

job_name_norm
ветеринарный врач                                                                                                                                                    212
агроном                                                                                                                                                              132
главный агроном                                                                                                                                                      114
механик                                                                                                                                                               92
главный ветеринарный врач                                                                                                                                             78
главный инженер                                                                                                                              

In [344]:
canon_perc = cfg["canon_job_names_top_percentile"]
apply_to_perc = cfg['try_to_find_canon_in_bottom_percentile']
if canon_perc < 0 or canon_perc > 100:
    print("Wrong canon_job_names_top_percentile value in config")
if apply_to_perc < 0 or apply_to_perc > 100:
    print("Wrong try_to_find_canon_in_bottom_percentile value in config")


value_counts = jobs_df['job_name'].value_counts()

top_quantive_value = 1 - canon_perc / 100
bottom_quantive_value = apply_to_perc / 100

# Определение порога для верхнего и нижнего 10-процентилей
top_percent_threshold = value_counts.quantile(top_quantive_value)
bottom_percent_threshold = value_counts.quantile(bottom_quantive_value)

# Получение значений для верхнего и нижнего 10-процентилей
bottom_percent_values = value_counts[value_counts <= bottom_percent_threshold].index
top_percent_values = set(value_counts[value_counts >= top_percent_threshold].index)

# Список строк, которые нужно удалить из top_10_percent_values
strings_to_remove = ['стажер', 'начальник', 'управляющий', 'глава', 'специалист']

# Удаление строк из top_10_percent_values
top_percent_values -= set(strings_to_remove)

print("Rows taken into top-percentile values: ", len(top_percent_values))
print("Rows taken into bottom-percentile values: ", len(bottom_percent_values))

# Функция для проверки, содержит ли строка из нижнего процентиля все слова из какой-либо строки верхнего процентиля
def contains_all_words_from_top(bottom_value, top_values) -> list[str]:
    all_matches = []
    bottom_words = set(bottom_value.split())
    for top_value in top_values:
        top_words = set(top_value.split())
        if top_words.issubset(bottom_words):
            all_matches.append(top_value)

    return all_matches

words_to_explicitly_check = ["старший", "главный", "помошник"]

# Итерация по строкам нижнего процентиля
for bottom_value in bottom_percent_values:
    all_matches = contains_all_words_from_top(bottom_value, top_percent_values)
    if all_matches:
        for i, match in enumerate(all_matches):
            for word in words_to_explicitly_check:
                if word in bottom_value and word not in match:
                    all_matches[i] = word + " " + match

    if not all_matches:
        continue

    # Выбрать лучший match из возможных (пока что по длинне строки)
    all_matches = sorted(all_matches, key=len, reverse=True)
    best_match = all_matches[0]
    if bottom_value != best_match:
        print(bottom_value + " -> " + best_match)
        print(all_matches)
        jobs_df['job_name_norm'] = jobs_df['job_name_norm'].replace(bottom_value, best_match)

Rows taken into top-percentile values:  133
Rows taken into bottom-percentile values:  2005
Машинист погрузчика 6 разряда Машинист-Эксковатора -> Машинист погрузчика
['Машинист погрузчика']
Машинист бульдозера 6 разряда-Машинист-Погрузчика-Машинист-Эксковатора -> Машинист бульдозера
['Машинист бульдозера']
Ветеринарный врач станции -> Ветеринарный врач
['Ветеринарный врач']
Ветеринарный врач ВСЭ -> Ветеринарный врач
['Ветеринарный врач']
Продавец-консультант ювелирных изделий -> Продавец-консультант
['Продавец-консультант']
Инженер по водоснабжению и канализации -> Инженер
['Инженер']
зоотехник селекционер, техник искусственного осеменения -> зоотехник
['зоотехник']
Лаборант по уходу за подопытными животными,  затем - лаборант гистологического отдела -> Лаборант
['Лаборант']
Мастер участка подготовки производства -> Мастер участка
['Мастер участка', 'Мастер']
Инженер по ремонту МТП -> Инженер
['Инженер']
зоотехник по кормлению -> зоотехник
['зоотехник']
Генеральный  директор торгового 

In [345]:
# Подключение к базе данных для сохранения сниппета
db_path_norm = "../db/normalized_data.db"
conn_norm = sqlite3.connect(db_path_norm)

jobs_df.to_sql(name='jobs', con=conn_norm, if_exists='replace', index=False)

4335